<a href="https://colab.research.google.com/github/msdsn/asl_fingerspelling/blob/main/Create_Tfr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create Tfr

In [1]:
import pandas as pd
from tqdm import tqdm

from joblib import Parallel, delayed
import multiprocessing as mp
from multiprocessing import cpu_count
from sklearn.model_selection import StratifiedGroupKFold, GroupKFold, KFold

import tensorflow as tf
cpu_count()

8

In [5]:
ls -l drive/MyDrive/KaggleDatasets/asl-fingerspelling/

total 10059
-rw------- 1 root root     405 May 25 23:24 character_to_prediction_index.json
drwx------ 2 root root    4096 Oct 21 13:04 supplemental_landmarks/
-rw------- 1 root root 5059429 May 25 23:24 supplemental_metadata.csv
-rw------- 1 root root 5231359 May 25 23:24 train.csv
drwx------ 2 root root    4096 Oct 21 12:34 train_landmarks/


````{tab-set}
```{tab-item} English
In the `train.csv` file, there is a `sequence_id` corresponding to each `phrase`, and the path to the parquet file is also specified. We need to match the phrase using the `sequence_id` value from that parquet file.
```

```{tab-item} Türkçe:
`train.csv` dosyası içerisinde her cümle (phrase) için karşılık gelen bir `sequence_id` bulunmaktadır ve ayrıca parquet dosyasının yolu da belirtilmiştir. Biz, bu parquet dosyasından alınan `sequence_id` değerini kullanarak cümleyi eşleştirmemiz gerekmektedir.
```
````

In [7]:
train_df = pd.read_csv('drive/MyDrive/KaggleDatasets/asl-fingerspelling/train.csv')
train_df

,path,file_id,sequence_id,participant_id,phrase
0,train_landmarks/5414471.parquet,5414471,1816796431,217,3 creekhouse
1,train_landmarks/5414471.parquet,5414471,1816825349,107,scales/kuhaylah
2,train_landmarks/5414471.parquet,5414471,1816909464,1,1383 william lanier
3,train_landmarks/5414471.parquet,5414471,1816967051,63,988 franklin lane
4,train_landmarks/5414471.parquet,5414471,1817123330,89,6920 northeast 661st road
...,...,...,...,...,...
67203,train_landmarks/2118949241.parquet,2118949241,388192924,88,431-366-2913
67204,train_landmarks/2118949241.parquet,2118949241,388225542,154,994-392-3850
67205,train_landmarks/2118949241.parquet,2118949241,388232076,95,https://www.tianjiagenomes.com
67206,train_landmarks/2118949241.parquet,2118949241,388235284,36,90 kerwood circle


````{tab-set}
```{tab-item} English
In the parquet files, there are multiple `sequence_id` attributes. For each `sequence_id`, there are several different frame data points. Within each frame, there is information on numerous landmarks, precisely 1,629 landmarks. These landmarks are defined by spatial coordinates in the format:
`[x/y/z]_[type]_[landmark_index]`

Here, the `[x/y/z]` represents the spatial coordinates, while `[type]` can be one of the following: 'face', 'left_hand', 'pose', or 'right_hand'. Lastly, `[landmark_index]` gives the specific index of the landmark among the total 543 landmarks.
```

```{tab-item} Türkçe:
parquet dosyalarında birçok `sequence_id` özelliği bulunmaktadır. Her `sequence_id` değeri için birçok farklı kare (frame) bilgisi mevcuttur. Her kare içerisinde, tam olarak 1,629 adet landmark (işaret noktası) bilgisi yer almaktadır. Bu işaret noktaları şu formatla tanımlanır:
`[x/y/z]_[tip]_[landmark_index]`

Burada, `[x/y/z]` mekansal koordinatları temsil ederken, `[tip]` şunlardan biri olabilir: 'yüz', 'sol_el', 'poz' veya 'sağ_el'. Son olarak, `[landmark_index]` toplam 543 işaret noktası arasındaki belirli işaret noktasının indeksini verir.
```
````

In [9]:
temp = pd.read_parquet('drive/MyDrive/KaggleDatasets/asl-fingerspelling//train_landmarks/1019715464.parquet')
temp

,frame,x_face_0,x_face_1,x_face_2,x_face_3,x_face_4,x_face_5,x_face_6,x_face_7,x_face_8,...,z_right_hand_11,z_right_hand_12,z_right_hand_13,z_right_hand_14,z_right_hand_15,z_right_hand_16,z_right_hand_17,z_right_hand_18,z_right_hand_19,z_right_hand_20
sequence_id,,,,,,,,,,,,,,,,,,,,,
1975433633,0,0.578892,0.578482,0.582906,0.572686,0.579030,0.582115,0.591430,0.498995,0.595773,...,-0.253687,-0.291687,-0.123892,-0.195255,-0.249135,-0.284375,-0.125050,-0.187797,-0.224827,-0.249662
1975433633,1,0.577563,0.578528,0.582916,0.572760,0.579090,0.582197,0.591687,0.497613,0.596104,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1975433633,2,0.576181,0.576949,0.581346,0.572293,0.577725,0.581191,0.591550,0.497113,0.596477,...,-0.233556,-0.267114,-0.116145,-0.179053,-0.223861,-0.253004,-0.122090,-0.173169,-0.200727,-0.219106
1975433633,3,0.575575,0.577569,0.581769,0.572443,0.578289,0.581652,0.591728,0.496759,0.596573,...,-0.190909,-0.218471,-0.093956,-0.149982,-0.188452,-0.211573,-0.102759,-0.147642,-0.168562,-0.180578
1975433633,4,0.577907,0.577628,0.582295,0.572930,0.578345,0.581873,0.592467,0.498170,0.597383,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008677531,70,0.684853,0.672752,0.679297,0.670935,0.673818,0.679804,0.697755,0.591439,0.707091,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008677531,71,0.683502,0.672031,0.678687,0.671496,0.673336,0.679910,0.699268,0.590744,0.709342,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008677531,72,0.688159,0.678438,0.683336,0.675476,0.679688,0.685293,0.701444,0.591098,0.710164,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
temp.index.value_counts()


1976808237    751
2003109217    709
1983592273    542
1994265137    521
2004864487    511
             ... 
1994754434      6
1991958149      6
1989533281      6
1999567111      6
1980910058      3
Name: sequence_id, Length: 998, dtype: int64